In [1]:
import gzip
import os
import json
import re
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from multiprocessing import Process
import time
import sys



In [2]:
names = [line.strip() for line in open('Ikon/extracted/all_galleries_raw.dat')]

In [3]:
for ind, n1 in enumerate(names):
    for n2 in names:

        #print(ind+1, '/408\t\t')

        if n1 != n2:
            fuzzy = fuzz.ratio(n1, n2)
            if fuzzy > 90:
                print(fuzzy, '\t', n1, n2)
                #fout.write(n1 + '\t' + n2 + '\t' + str(fuzzy) + '\n')



98 	 Art Factory Galéria I. Art Factory Galéria II.
98 	 Art Factory Galéria II. Art Factory Galéria I.
91 	 Bartók 1 Galéria Bartók 32 Galéria
91 	 Bartók 32 Galéria Bartók 1 Galéria
91 	 Budapest Galéria Új Budapest Galéria
92 	 Béka Kortárs Művészeti Galéria acb Kortárs Művészeti Galéria
95 	 Galéria IX Galéria X
95 	 Galéria X Galéria IX
92 	 Miskolci Kortárs Művészeti Intézet (M.ICA) Miskolci Kortárs Művészeti Intézet (M.ICA) - ZOOM
92 	 Miskolci Kortárs Művészeti Intézet (M.ICA) - ZOOM Miskolci Kortárs Művészeti Intézet (M.ICA)
94 	 OFF-Biennálé 2015 OFF-Biennálé 2017
94 	 OFF-Biennálé 2017 OFF-Biennálé 2015
92 	 acb Kortárs Művészeti Galéria Béka Kortárs Művészeti Galéria
96 	 Érdi Galéria Éri Galéria
96 	 Éri Galéria Érdi Galéria
91 	 Új Budapest Galéria Budapest Galéria


## matching ikon magnus -- ez nem igazán érdekes

In [4]:
set(pd.read_csv('../../Hungarian Art Data Project-2020/Data/BarabasiLabData_HungaryInstitutionInfo.csv').Type)

{'gallery', 'museum'}

In [20]:
names1 = [line.strip() for line in open('Ikon/extracted/all_galleries_raw.dat')]
names2 = list(pd.read_csv('../../Hungarian Art Data Project-2020/Data/BarabasiLabData_HungaryInstitutionInfo.csv').Name)

off = ['Galéria', 'Galeria', 'Gallery', 'Múzeum', 'Museum']
n = 0


fout = open('IkonMatchingMagnus.csv', 'w')
fout.write('Ikon\tMagnus\tSimilarity\n')

for n1 in names1:

    line  = n1 + '\t'
    match = ('', 0)
    
    for n2 in names2:
        
        n11 = n1
        n22 = n2
        
        for o in off:    
            n11 = n11.replace(o, '').lower()
            n22 = n22.replace(o, '').lower()
        
        fuzzy = fuzz.ratio(n11, n22)
        if fuzzy > 69:
            if fuzzy > match[1]:
                match = (n2, fuzzy)

        
    if match[1] > 0:
        line = line + match[0] + '\t' + str(match[1]) + '\n'
        n+=1
    else:
        line = line + 'unmatched\tna\n'
        
    fout.write(line)
       
fout.close()        
print(n)


df = pd.read_csv('IkonMatchingMagnus.csv', sep = '\t')
df = df.sort_values(by = 'Similarity')
df.to_excel('../../Hungarian Art Data Project-2020/Data/IkonMatchingMagnus.xls')

51


## matching ikon magnus -- ennek a kimenetel tábláját (Dropbox...) használja Anna

In [42]:
fout = open('MagnusMatchinIkon.csv', 'w')
fout.write('Magnus\tIkon\tSimilarity\n')

n = 0


for n1 in names2:
    
    
    line = n1 + '\t'
    
    match = ('', 0)
    
    for n2 in names1:
        
        n11 = n1
        n22 = n2
        
        for o in off:    
            n11 = n11.replace(o, '').lower()
            n22 = n22.replace(o, '').lower()
        
        
        fuzzy = fuzz.ratio(n11, n22)
        if fuzzy > 69:
            #print(fuzzy, '\t', n1, '--\t--', n2)
            if fuzzy > match[1]:
                match = (n2, fuzzy)
            #
                
        #else:
            #
    
    #print(match)
        
    if match[1] > 0:
        line = line + match[0] + '\t' + str(match[1]) + '\n'
        n += 1
    else:
        line = line + 'unmatched\t-1\n'
        
    fout.write(line)
       
fout.close()        
print(n)

df = pd.read_csv('MagnusMatchinIkon.csv', sep = '\t')

df.index = df.Magnus
df = df.drop(columns = ['Magnus'])

df.Similarity = df.Similarity.astype(int)
df = df.sort_values(by = 'Similarity', ascending = False)
df.to_excel('../../Hungarian Art Data Project-2020/Data/MagnusMatchinIkon.xls')
df.head()

58


,Ikon,Similarity
Magnus,,
Budapest Galéria,Budapest Galéria,100
Impex,IMPEX,100
Nessim Galéria,Nessim Galéria,100
Hal Köz Galéria,Hal Köz Galéria,100
Karinthy Szalon,Karinthy Szalon,100


In [44]:
df2 = pd.read_csv('../../Hungarian Art Data Project-2020/Data/IkonGalleryNamesRaw.txt')
df2 = df2.sort_values(by = 'name')#, ascending = False)
df2.index = df2.name
df2 = df2.drop(columns = ['name'])
df2.to_excel('../../Hungarian Art Data Project-2020/Data/IkonGalleryNameFrequency.xls')